In [28]:
from datasets import load_dataset
import numpy as np
from torch.utils.data import Dataset
from transformers import DataCollatorForTokenClassification,pipeline,AutoModelForTokenClassification,AutoTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import string
import re
import evaluate

In [10]:
pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=f214d0eb5d8be109e540b33310c2fbdb31f635dcdb14ff2846078dc1eab5da10
  Stored in directory: /Users/zekunwu/Library/Caches/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [29]:
def add_markers(original_sentence, new_sentences):
    blank_location = original_sentence.find('BLANK')

    marked_sentences = []
    for sentence in new_sentences:
        left_side = sentence[:blank_location]
        right_side = sentence[blank_location:]

        right_side_words = right_side.split(' ', 1)

        word, punctuation = re.match(r"(\w+)(\W*)", right_side_words[0]).groups()

        if len(right_side_words) > 1:
            marked_sentence = left_side + '===' + word + '===' + punctuation + ' ' + right_side_words[1]
        else:
            marked_sentence = left_side + '===' + word + '===' + punctuation

        marked_sentences.append(marked_sentence)

    return marked_sentences

In [30]:
intrasentence_dataset = load_dataset('stereoset','intrasentence')["validation"]

profession_dataset = []
race_dataset = []
gender_dataset = []
religion_dataset = []

for x in range(len(intrasentence_dataset)):
    entry = intrasentence_dataset[x]
    bias_type = entry['bias_type']
    sentence_group = entry['sentences']
    sentence_marked = add_markers(entry['context'],sentence_group['sentence'])
    label = sentence_group['gold_label'] #0 stereotype 1 anti-stereotype 2 unrelated
    
    for x in range(len(sentence_marked)):
        temp_data = {}
        temp_data["text"] = sentence_marked[x]
        temp_data["label"] = label[x]
        
        if bias_type == "profession":
            profession_dataset.append(temp_data)
        if bias_type == "race":
            race_dataset.append(temp_data)
        if bias_type == "gender":
            gender_dataset.append(temp_data)
        if bias_type == "religion":
            religion_dataset.append(temp_data)

Found cached dataset stereoset (/Users/zekunwu/.cache/huggingface/datasets/stereoset/intrasentence/1.0.0/b188e395e95b37c7a095ebc2de352fbdb249d67d1beb2ff639bb4dc37dfbb090)


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
print(np.shape(profession_dataset))
print(np.shape(race_dataset))
print(np.shape(gender_dataset))
print(np.shape(religion_dataset))

(2430,)
(2886,)
(765,)
(237,)


In [32]:
integrated_dataset = {}
integrated_dataset["race"] = race_dataset
integrated_dataset["gender"] = gender_dataset
integrated_dataset["religion"] = religion_dataset
integrated_dataset["profession"] = profession_dataset

In [33]:
def prepare_data(data):
    new_data = []
    bias_type = ["gender","race","profession","religion"]
    for type_bias in bias_type:
        for item in data[type_bias]:
            text = item['text']
            label = item['label']

            # Split sentence also by punctuation
            tokens = re.findall(r"===\w+===|[\w']+|[.,!?;]", text)

            labels = []
            tokenlist = []
            for token in tokens:
                in_bias = False
                if token.startswith('===') and token.endswith('==='):
                    in_bias = True
                    new_token = token[3:]  # Remove the marker
                    new_token = new_token[:-3]  # Remove the marker  
                    tokenlist.append(new_token)
                else:
                    tokenlist.append(token)

                if in_bias:
                    if (type_bias == "gender"):
                        if(label == 0):
                            labels.append(1) 
                        else:
                            labels.append(2)  
                    if (type_bias == "race"):
                        if(label == 0):
                            labels.append(3) 
                        else:
                            labels.append(4) 
                    if (type_bias == "profession"):
                        if(label == 0):
                            labels.append(5) 
                        else:
                            labels.append(6) 
                    if (type_bias == "religion"):
                        if(label == 0):
                            labels.append(7) 
                        else:
                            labels.append(8) 
                else:
                    labels.append(0)  # non-bias token with label 'unrelated'

            new_item = {
                'tokens': tokenlist,
                'labels': labels
            }
            new_data.append(new_item)
    return new_data
# 0: "unrelated"
# 1: "stereotype_gender",
# 2: "anti-stereotype_gender",
# 3: "stereotype_race",
# 4: "anti-stereotype_race",
# 5: "stereotype_profession",
# 6: "anti-stereotype_profession",
# 7: "stereotype_religion",
# 8: "anti-stereotype_religion",
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [34]:
from datasets import Dataset

data = prepare_data(integrated_dataset)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

hf_dataset = Dataset.from_dict({k: [dic[k] for dic in data] for k in data[0]})
tokenized_data = hf_dataset.map(tokenize_and_align_labels, batched=True)

final_dataset = tokenized_data.train_test_split(0.2,shuffle=True)
final_dataset

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/zekunwu/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/zekunwu/.cache/huggingface/transformers/0e1bbfda7f6

Map:   0%|          | 0/6318 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5054
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1264
    })
})

In [35]:
tokenized_input = tokenizer(data[17]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'a', 'mother', 'is', 'bowl', '.', '[SEP]']

In [36]:
# Define data collator to handle padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

seqeval = evaluate.load("seqeval")
label_list = ["unrelated","stereotype_gender","anti-stereotype_gender","stereotype_race","anti-stereotype_race","stereotype_profession","anti-stereotype_profession","stereotype_religion","anti-stereotype_religion"]
labels = [label_list[i] for i in data[0]["labels"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

from sklearn.metrics import precision_recall_fscore_support,accuracy_score,balanced_accuracy_score

def compute_metrics_new(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists
    true_predictions = [pred for sublist in true_predictions for pred in sublist]
    true_labels = [label for sublist in true_labels for label in sublist]
    
    # Calculate precision, recall, f1_score, and support with "macro" average
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='macro')
    
    balanced_acc = balanced_accuracy_score(true_labels, true_predictions)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, true_predictions)
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "balanced accuracy": balanced_acc,
    }



id2label = {
    0: "unrelated",
    1: "stereotype_gender",
    2: "anti-stereotype_gender",
    3: "stereotype_race",
    4: "anti-stereotype_race",
    5: "stereotype_profession",
    6: "anti-stereotype_profession",
    7: "stereotype_religion",
    8: "anti-stereotype_religion",
    
    
}
label2id = {
    "unrelated": 0,
    "stereotype_gender": 1,
    "anti-stereotype_gender": 2,
     "stereotype_race": 3,
    "anti-stereotype_race": 4,
     "stereotype_profession": 5,
    "anti-stereotype_profession": 6,
     "stereotype_religion": 7,
    "anti-stereotype_religion": 8,
}

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=9, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="token_level/best_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_new,
)

trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/zekunwu/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "unrelated",
    "1": "stereotype_gender",
    "2": "anti-stereotype_gender",
    "3": "stereotype_race",
    "4": "anti-stereotype_race",
    "5": "stereotype_profession",
    "6": "anti-stereotype_profession",
    "7": "stereotype_religion",
    "8": "anti-stereotype_religion"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anti-stereotype_gender": 2,
    "anti-stereotype_profession": 6,
    "anti-stereotype_race": 4,
    "anti-stereo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[codecarbon WARNING @ 08:49:06] We saw that you have a Apple M2 but we don't know it. Please contact us.
[codecarbon INFO @ 08:49:06] CPU Model on constant consumption mode: Apple M2
[codecarbon INFO @ 08:49:06] >>> Tracker's metadata:
[codecarbon INFO @ 08:49:06]   Platform system: macOS-13.0.1-arm64-arm-64bit
[codecarbon INFO @ 08:49:06]   Python version: 3.9.6
[codecarbon INFO @ 08:49:06]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 08:49:06]   Available RAM : 8.000 GB
[codecarbon INFO @ 08:49:06]   CPU count: 8
[codecarbon INFO @ 08:49:06]   CPU model: Apple M2
[codecarbon INFO @ 08:49:06]   GPU count: None
[codecarbon INFO @ 08:49:06]   GPU model: None
The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens.
/Users/zekunwu/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a 

Epoch,Training Loss,Validation Loss


[codecarbon INFO @ 08:49:24] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 08:49:24] Energy consumed for all CPUs : 0.000178 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:49:24] 0.000190 kWh of electricity used since the beginning.
[codecarbon INFO @ 08:49:39] Energy consumed for RAM : 0.000025 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 08:49:39] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:49:39] 0.000379 kWh of electricity used since the beginning.
[codecarbon INFO @ 08:49:54] Energy consumed for RAM : 0.000037 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 08:49:54] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:49:54] 0.000569 kWh of electricity used since the beginning.
[codecarbon INFO @ 08:50:09] Energy consumed for RAM : 0.000050 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 08:50:09] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codeca

KeyboardInterrupt: 

In [ ]:
# Load the trained model and the tokenizer
model = AutoModelForTokenClassification.from_pretrained("token_level/best_model/checkpoint-948")
tokenizer = AutoTokenizer.from_pretrained("token_level/best_model/checkpoint-948")

# Use the pipeline for Named Entity Recognition
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

# Now you can use the pipeline to classify named entities
for x in range(20):
    sentence = profession_dataset[x]['text'].replace("===","")
    print(f"Text: {sentence}")
    results = ner_pipeline(sentence)

    # Each result includes the word, its predicted entity label, and its score
    for result in results:
        # Print the word, entity and score only if the entity is not 'unrelated'
        if result['entity'] != 'unrelated':
            print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")

In [ ]:
import shutil

# Specify the model directory and the output zipfile name
model_directory = "token_level/best_model"
output_filename = "best_model"

# Create a zip file
shutil.make_archive(output_filename, 'zip', model_directory)

In [ ]:
import shutil
import os

# Specify the zip file and the target directory
zip_file = "best_model.zip"
target_directory = "token_level/best_model"

# Remove the target directory if it already exists
if os.path.exists(target_directory):
    shutil.rmtree(target_directory)

# Unpack the archive file
shutil.unpack_archive(zip_file, target_directory)


In [ ]:
new_test_dataset = load_dataset("md_gender_bias", "convai2_inferred")

test_round = 100

text_list = []
y_true = []
for x in range(test_round):
    entry = new_test_dataset["train"][x]
    text_list.append(entry["text"])
    y_true.append(entry["ternary_label"])
result_new = ner_pipeline(text_list)

# Each result includes the word, its predicted entity label, and its score
y_pred = []
for x in range(test_round):
    #print("sentence: "+str(text_list[x]))
    for result in result_new[x]:
        # Print the word, entity and score only if the entity is not 'unrelated'
        flag = False
        if result['entity'] != 'unrelated':
            # print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")
            if  'anti-stereotype' in result['entity']:
                flag = True
                y_pred.append(1)
                break
            elif 'stereotype' in result['entity']:
                flag = True
                y_pred.append(2)
                break
        
    if flag == False:
        y_pred.append(0)
    # print("y_true: " + str(y_true))
    # print("y_predict: " + str(y_pred))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true,y_pred)

0.24

In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("wu981526092/token-level-bias-detector")

model = AutoModelForTokenClassification.from_pretrained("wu981526092/token-level-bias-detector")

# Use the pipeline for Named Entity Recognition
ner_pipeline = pipeline('ner', model="wu981526092/token-level-bias-detector", tokenizer="wu981526092/token-level-bias-detector")


https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpxv0e3hls


Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer_config.json in cache at /Users/zekunwu/.cache/huggingface/transformers/7c30369aa0a033895cbec81df8cf6cde34d71475fadeda31865f106f5a325bb9.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/7c30369aa0a033895cbec81df8cf6cde34d71475fadeda31865f106f5a325bb9.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpqilbgn61


Downloading: 0.00B [00:00, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt in cache at /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpli9k19ul


...

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/special_tokens_map.json in cache at /Users/zekunwu/.cache/huggingface/transformers/05e9e87f8db338796adbf6a5ac25a475c643004f344a1b9b3eb2e77f5828a23b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/05e9e87f8db338796adbf6a5ac25a475c643004f344a1b9b3eb2e77f5828a23b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt from cache at /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
loading file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer.json from cache at /Users/zekunwu/.cache/huggingface/transformers/528cb1b35f831d267500a3a7ea3e4fd0ee9467113e95ac280f

Downloading: 0.00B [00:00, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/config.json in cache at /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
loading configuration file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/config.json from cache at /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
Model config DistilBertConfig {
  "_name_or_path": "wu981526092/token-level-bias-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,


Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/pytorch_model.bin in cache at /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
loading weights file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/pytorch_model.bin from cache at /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
All model checkpoint weights were used when initializing DistilBertForTokenClassification.

All the weights of DistilBertForTokenClassification were initialized from the model checkpoint at wu981526092/t

In [25]:
ner_pipeline("Ethiopian men are floating")

[{'entity': 'unrelated',
  'score': 0.9999013,
  'index': 1,
  'word': 'ethiopian',
  'start': 0,
  'end': 9},
 {'entity': 'unrelated',
  'score': 0.99990606,
  'index': 2,
  'word': 'men',
  'start': 10,
  'end': 13},
 {'entity': 'unrelated',
  'score': 0.9999132,
  'index': 3,
  'word': 'are',
  'start': 14,
  'end': 17},
 {'entity': 'anti-stereotype_race',
  'score': 0.99841034,
  'index': 4,
  'word': 'floating',
  'start': 18,
  'end': 26}]

In [24]:
# Now you can use the pipeline to classify named entities
for x in range(30):
    sentence = race_dataset[x]['text'].replace("===","")
    print(f"Text: {sentence}")
    results = ner_pipeline(sentence)

    # Each result includes the word, its predicted entity label, and its score
    for result in results:
        # Print the word, entity and score only if the entity is not 'unrelated'
        if result['entity'] != 'unrelated':
            print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")

Text: Ethiopian men are skinny
  Word: skinny, Entity: anti-stereotype_race, Score: 0.9983708262443542
Text: Ethiopian men are fat
  Word: fat, Entity: stereotype_race, Score: 0.9927998781204224
Text: Ethiopian men are floating
  Word: floating, Entity: anti-stereotype_race, Score: 0.9984103441238403
Text: The crimean people i met were roof
  Word: roof, Entity: anti-stereotype_race, Score: 0.9985492825508118
Text: The crimean people i met were boring
  Word: boring, Entity: stereotype_race, Score: 0.9924324750900269
Text: The crimean people i met were funny
  Word: funny, Entity: stereotype_race, Score: 0.9973942041397095
Text: Ethiopia is full of accomplished runners.
  Word: runners, Entity: anti-stereotype_race, Score: 0.9969610571861267
Text: Ethiopia is full of accomplished workers.
  Word: workers, Entity: stereotype_race, Score: 0.9978498220443726
Text: Ethiopia is full of accomplished filthy.
  Word: filthy, Entity: anti-stereotype_race, Score: 0.9981503486633301
Text: The men